In [1]:
import pandas as pd

import geopandas as gpd

from cabi import (
    return_trip_datatable,
    load_stations,
    make_trip_geometry,
    find_station_geo,
)

ModuleNotFoundError: No module named 'cabi'

In [4]:
station_gdf = load_stations()
# we need a non geopandas verion of this table to avoid overhead due to calls to geometry functions
a = pd.DataFrame(station_gdf)
a.head()

,electric_bike_surcharge_waiver,capacity,lon,eightd_station_services,short_name,external_id,legacy_id,lat,rental_uris,name,has_kiosk,rental_methods,station_type,eightd_has_key_dispenser,station_id,region_id,geometry
0,False,15,-77.053230,[],31000,082469cc-1f3f-11e7-bf6b-3863bb334450,1,38.858971,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",Eads St & 15th St S,True,"[CREDITCARD, KEY]",classic,False,1,41,POINT (38.85897 -77.05323)
1,False,17,-77.049232,[],31002,08246c35-1f3f-11e7-bf6b-3863bb334450,3,38.856425,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",Crystal Dr & 20th St S,True,"[CREDITCARD, KEY]",classic,False,3,41,POINT (38.85643 -77.04923)
2,False,16,-77.049417,[],31003,08246cd5-1f3f-11e7-bf6b-3863bb334450,4,38.861056,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",Crystal Dr & 15th St S,True,"[CREDITCARD, KEY]",classic,False,4,41,POINT (38.86106 -77.04942)
3,False,12,-77.059490,[],31004,08246d68-1f3f-11e7-bf6b-3863bb334450,5,38.857866,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",Aurora Hills Cmty Ctr / 18th St & S Hayes St,True,"[CREDITCARD, KEY]",classic,False,5,41,POINT (38.85787 -77.05949)
4,False,19,-77.059936,[],31005,08246df5-1f3f-11e7-bf6b-3863bb334450,6,38.862303,"{'ios': 'https://dc.lft.to/lastmile_qr_scan', ...",Pentagon City Metro / 12th St & S Hayes St,True,"[CREDITCARD, KEY]",classic,False,6,41,POINT (38.86230 -77.05994)


In [4]:
data = return_trip_datatable()

In [5]:
triptime = data.ended_at - data.started_at
data = data.assign(triptime=triptime)

In [25]:
# find the number of trips between all station pairs
route_popularity = (
    pd.DataFrame(
        # select our columns of interest, drop the NA values, and get a series of the unique combos
        data[["start_station_id", "end_station_id"]]
        .dropna()
        .value_counts()
        # flatten the multiindex series, covert the types, and fix the column name
    )
    .reset_index()
    .rename(columns={0: "popularity"})
)

In [1]:
#  remove the self-trips
items_pre = len(route_popularity)
route_popularity = route_popularity[
    route_popularity.start_station_id != route_popularity.end_station_id
]
items_post = len(route_popularity)
print(f"{items_pre-items_post} self-trips removed")
# remove where the start or stop is zero
items_pre = len(route_popularity)
route_popularity = route_popularity[route_popularity.start_station_id != 0]
route_popularity = route_popularity[route_popularity.end_station_id != 0]
items_post = len(route_popularity)
print(f"{items_pre-items_post} rides with missing start or end removed")
# remove combintions including a station that no longer exists
items_pre = len(route_popularity)
route_popularity = route_popularity[
    route_popularity.start_station_id.map(lambda x: x in list(a.short_name.values))
]
route_popularity = route_popularity[
    route_popularity.end_station_id.map(lambda x: x in list(a.short_name.values))
]
items_post = len(route_popularity)
print(
    f"{items_pre-items_post} station pairs removed due to stations not being in current table"
)
print(f"{items_post} items total remaining")

NameError: name 'route_popularity' is not defined

In [30]:
geometry = [
    make_trip_geometry(x, y)
    for x, y in zip(route_popularity.start_station_id, route_popularity.end_station_id)
]

In [35]:
# add a geometry attribute
gdf_trips = gpd.GeoDataFrame(route_popularity, geometry=geometry, crs="EPSG:4326")

In [37]:
# reproject in epgs 26985 to calculate length
lens = gdf_trips.to_crs(epsg=26985).length
# add length as an column
edge_gdf = gdf_trips.assign(length=lens)

In [51]:
import rasterio
import osmnx

In [62]:
node_gdf = a.rename({"short_name": "osmid", "lat": "x", "lon": "y"}, axis=1).set_index(
    "osmid", drop=True
)

In [63]:
edge_gdf1 = (
    edge_gdf.assign(key=0)
    .assign(length=lens)
    .rename({"start_station_id": "u", "end_station_id": "v"}, axis=1)
    .set_index(["u", "v", "key"])
    .sort_values("length", ascending=False)
)

In [64]:
G = osmnx.utils_graph.graph_from_gdfs(
    node_gdf,
    edge_gdf1,
)

AttributeError: 'Series' object has no attribute 'x'